In [1]:
# ============================================================
# 01 - Importación de librerías
# ============================================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os, time, sys, glob, shutil, json


# ============================================================
# 02 - Cargar configuración desde JSON (credenciales + URLs)
# ============================================================
CONFIG_PATH = "ConfigWBBR.json"

if not os.path.isfile(CONFIG_PATH):
    raise FileNotFoundError(f"[ERROR] No se encontró el archivo {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

USER = config["USER"]
PWD  = config["PWD"]
LOGIN_URL = config["LOGIN_URL"]
CSPS_URL  = config["CSPS_URL"]


# ============================================================
# 03 - Configuración de rutas de archivos
# ============================================================
PARTS_DIR  = os.path.abspath(r"C:\partes_txt")       # Carpeta con .txt de entrada
OUTPUT_DIR = os.path.abspath(r"C:\descargas\BR")     # Carpeta de salida para los .xls


# ============================================================
# 04 - Configuración del navegador (Chrome)
# ============================================================
options = Options()
options.add_argument("--start-maximized")
prefs = {"download.default_directory": OUTPUT_DIR}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)


# ============================================================
# 05 - Helpers (funciones auxiliares reutilizables)
# ============================================================
def safe_find(by, selector, timeout=30):
    """Encuentra un elemento y espera hasta que esté presente en el DOM."""
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((by, selector))
    )

def safe_click(by, selector, timeout=30):
    """Hace clic en un elemento asegurándose de que es clickable."""
    el = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by, selector))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    el.click()
    return el

def switch_to_frame(name, timeout=30):
    """Cambia al frame especificado."""
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.NAME, name))
    )

def back_to_default():
    """Regresa al contenido principal fuera de los frames."""
    driver.switch_to.default_content()

def get_latest_download(path=OUTPUT_DIR, ext="xls"):
    """Obtiene el archivo más reciente descargado en OUTPUT_DIR."""
    files = glob.glob(os.path.join(path, f"*.{ext}"))
    if not files:
        return None
    return max(files, key=os.path.getctime)


# ============================================================
# 06 - Flujo de navegación
# ============================================================
def login_and_open_csps_latam():
    """Login en portal CNH y abrir CSPS LATAM en nueva pestaña."""
    driver.get(LOGIN_URL)
    safe_find(By.ID, "userID", 20).send_keys(USER)
    driver.find_element(By.ID, "password").send_keys(PWD)
    driver.find_element(By.ID, "btn-submit").click()

    # Esperar a que pase a my.dlrportal.com
    wait.until(EC.url_contains("my.dlrportal.com"))

    # Abrir CSPS LATAM en nueva pestaña
    driver.switch_to.new_window("tab")
    driver.get(CSPS_URL)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "frameset")))

def open_consulta_multiple_stock():
    """Menú -> Consulta de Referencias -> Consulta Múltiple de Stock."""
    back_to_default()
    switch_to_frame("menu", 20)

    # 1) Expandir el folder "Consulta de Referencias"
    safe_click(By.ID, "folderIcon9", 20)
    time.sleep(1)

    # 2) Click en "Consulta Múltiple de Stock"
    link = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//a[@title='Consulta Múltiple de Stock']"))
    )
    driver.execute_script("arguments[0].click();", link)

    # 3) Cambiar al frame de detalle
    back_to_default()
    switch_to_frame("detail", 20)
    safe_find(By.ID, "filePath", 20)


# ============================================================
# 07 - Main (ejecución principal)
# ============================================================
try:
    if not os.path.isdir(PARTS_DIR):
        print(f"[ERROR] No existe carpeta de .txt: {PARTS_DIR}")
        sys.exit(1)

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    login_and_open_csps_latam()
    open_consulta_multiple_stock()

    for fname in sorted(os.listdir(PARTS_DIR)):
        if not fname.endswith(".txt"):
            continue

        file_path = os.path.join(PARTS_DIR, fname)
        print(f"[INFO] Subiendo {file_path}")

        # Subir archivo
        file_input = safe_find(By.ID, "filePath", 20)
        file_input.send_keys(file_path)

        # Clic en Consultar
        safe_click(By.NAME, "Search", 20)

        # Esperar resultados
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//td[contains(., 'Referencia')]")
        ))
        time.sleep(2)

        # Descargar Excel
        safe_click(By.NAME, "Download", 20)
        print(f"[OK] Descargando resultado de {fname}")

        # Renombrar archivo descargado
        time.sleep(6)
        latest = get_latest_download()
        if latest:
            new_name = os.path.splitext(fname)[0] + ".xls"
            new_path = os.path.join(OUTPUT_DIR, new_name)
            try:
                shutil.move(latest, new_path)
                print(f"[OK] Guardado como {new_path}")
            except Exception as e:
                print(f"[WARN] No se pudo renombrar {latest}: {e}")

    print("[FIN] Todas las partes procesadas.")

except Exception as e:
    print("[ERROR]", repr(e))
finally:
    try:
        input("ENTER para cerrar...")
    except:
        pass
    driver.quit()


[INFO] Subiendo C:\partes_txt\part1.txt
[OK] Descargando resultado de part1.txt
[OK] Guardado como C:\descargas\BR\part1.xls
[INFO] Subiendo C:\partes_txt\part10.txt
[OK] Descargando resultado de part10.txt
[OK] Guardado como C:\descargas\BR\part10.xls
[INFO] Subiendo C:\partes_txt\part11.txt
[ERROR] NoSuchWindowException()
